In [29]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras import backend as k
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import numpy as np
import h5py
import os
import cv2

In [12]:
img_width, img_height = 256, 256

### Build the network
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(256, 256, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block3_pool'))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block4_pool'))

# TODO Use this block ?
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block5_pool'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
__________

In [13]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
a = [layer.name for layer in model.layers]
print(a)

['block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']


In [14]:
weights_path = './vgg19_weights.h5' 
# ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)
model.load_weights(weights_path, by_name=True)

layer_count=0;
weights_dict = []
for layer in model.layers:
    weights = layer.get_weights()
    weights_dict.append(weights)
    layer_count = layer_count + 1
    print("Model Layer name : {}".format(str(layer.name)))

print("The total number layers is : " + str(layer_count))
for i in layer_dict.keys():
    index = a.index(i)
    model.layers[index].set_weights(weights_dict[index])

Model Layer name : block1_conv1
Model Layer name : block1_conv2
Model Layer name : block1_pool
Model Layer name : block2_conv1
Model Layer name : block2_conv2
Model Layer name : block2_pool
Model Layer name : block3_conv1
Model Layer name : block3_conv2
Model Layer name : block3_conv3
Model Layer name : block3_pool
Model Layer name : block4_conv1
Model Layer name : block4_conv2
Model Layer name : block4_conv3
Model Layer name : block4_pool
Model Layer name : block5_conv1
Model Layer name : block5_conv2
Model Layer name : block5_conv3
Model Layer name : block5_pool
The total number layers is : 18


In [5]:

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
__________

In [25]:
X_train = []
y_train = []

dataset_path = "./data"

directories = [f for f in os.listdir(dataset_path)]

print(directories)

['user_10', 'user_3', 'user_4', 'user_5', 'user_6', 'user_7', 'user_9']


In [41]:
label = 0
for direc in directories:
    # For each class located in differents folders
    print("Label {} : {}".format(label, direc))
    for file in os.listdir(dataset_path + "/" + direc):
        try:
            path = os.path.join(dataset_path,direc,file)
            img = cv2.imread(path, 0)
            # BRG from openCV to RGB
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            img = img.astype(np.float32) / 255.0 - 0.5
            X_train.append(img)
            y_train.append(label)
        except Exception as e:
            print("Error with file : {}".format(file))
            print(e)
    label += 1

Label 0 : user_10
Error with file : user_10_loc.csv
/Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize

Label 1 : user_3
Error with file : user_3_loc.csv
/Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize

Label 2 : user_4
Error with file : user_4_loc.csv
/Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize

Label 3 : user_5
Error with file : user_5_loc.csv
/Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize

Label 4 : user_6
Error with file : user_6_loc.csv
/Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.heig

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
X_train = np.array(X_train)
shift = 0.2
rotation_range = 20

# Train generator
datagen = ImageDataGenerator(rotation_range=rotation_range, width_shift_range=shift, height_shift_range=shift)
datagen.fit(X_train)

# Validation generator
datagen_validation = ImageDataGenerator()
datagen_validation.fit(X_train)

In [ ]:
print(np.array(X_train).shape)
print(np.array(y_train).shape)
y_train = np.array(y_train)

y_train_one_hot = np.eye(max(y_train + 1))[y_train]

X_train, y_train_one_hot = shuffle(X_train, y_train_one_hot)

In [ ]:
batch_size = 16
adam = Adam(lr=0.001)
model.compile(loss='mse', optimizer=adam, metrics=['accuracy'])

filepath="./models_vgg/checkpoint_model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history_object = model.fit_generator(datagen.flow(X_train, y_train_one_hot, batch_size=batch_size),
                                     validation_data=datagen_validation.flow(X_train, y_train_one_hot, batch_size=32),
                                     samples_per_epoch=len(X_train),
                                     epochs=60,
                                     verbose=1,
                                     validation_steps=int(len(X_train) / batch_size),
                                     callbacks=callbacks_list)

In [ ]:
model_path = 'models_vgg/first_model.h5'
model.save(model_path)
print("Model saved to {}".format(model_path))